In [9]:
# импортирую библиотеки
import pandas as pd
from tqdm import tqdm

In [26]:
# загружаю ГБД ФЛ в датафрейм
df = pd.read_csv('data/inter/gbd_fl.csv', index_col=0, parse_dates=['birth_date', 'death_date'])
df

,birth_date,sex,nationality,citizenship,death_date,life_status,birth_country,birth_oblast,birth_rayon,birth_town,address_country,address_oblast,address_rayon,address_town,address_temp_country,address_temp_oblast,address_temp_rayon,address_temp_town,work_status,lifetime
iin,,,,,,,,,,,,,,,,,,,,
0,1937-07-17,2,3,1,2020-12-06,2,1,12,15,3,1,12,15,3,1,12,15,3,1,30458.0
1,1958-09-18,1,1,1,2002-02-18,2,1,5,9,1,1,5,9,1,1,5,9,1,1,15859.0
2,1955-01-20,1,1,1,2021-05-16,2,1,13,6,6,1,13,6,6,1,13,6,6,2,24223.0
3,2004-06-18,1,1,3,NaT,1,1,3,7,8,1,3,7,8,1,3,7,8,2,NaN
4,1927-02-18,1,1,1,1992-02-14,2,3,6,13,2,3,6,13,2,3,6,13,2,2,23737.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1990-01-19,2,1,1,NaT,1,1,7,18,6,1,7,18,6,1,7,18,6,2,NaN
499996,1921-06-22,1,2,3,1949-10-02,2,1,11,15,9,1,11,15,9,1,11,15,9,1,10329.0
499997,1960-10-28,1,1,1,2010-03-22,2,1,7,19,6,1,7,19,6,1,7,19,6,2,18042.0


## Младенческая и детская смертность

Методы расчета коэффициента младенческой смертности (смертность детей в возрасте до 1 года) отличаются от методов расчета всех других возрастных коэффициентов. При вычислении число смертей детей в возрасте до 1 года делится не на их среднегодовую численность, а на число родившихся.

1. Коэффициент мертворождаемости рассчитывается как отношение числа мертворожденных в данном году к числу родившихся живыми и мертвыми в том же году.

2. Коэффициент перинатальной смертности рассчитывается как отношение числа мертворожденных, а также умерших в первые 7 суток жизни к общему числу родившихся живыми и мертвыми.

3. Неонатальная смертность отражает смертность детей с момента рождения до 28 дня жизни. Коэффициент неонатальной смертности рассчитывается как отношение числа умерших в неонатальном периоде  к числу родившихся живыми в том же году.

4. Коэффициент ранней неонатальной смертности (смертность в течение первых 7 дней жизни) определяется отношением числа умерших в раннем неонатальном периоде  к числу родившихся живыми в том же году.

5. Коэффициент поздней неонатальной смертности (смертность, имеющая место в период после 7 дней жизни до 28 дней жизни) определяется отношением числа умерших в позднем неонатальном периоде к числу родившихся живыми в том же году.

In [29]:
def infant_mortality(df, start_year, end_year):
    """
    Входные данные:
        df: датафрейм с физлицами
        start_year: год начала интервала подсчета коэффициентов
        end_year: год конца интервала подсчета коэффициентов
    Выходные данные:
        df_infant_mortality: датафрейм содержащий коэффициенты
        мертворождаемости, перинатальной, неонатальной, ранней
        неонатальной, и поздней неонатальной смертности в годы
        по start_year до end_year
    """
    years = range(start_year, end_year)
    rates = {
        'stillbirth':      dict(),
        'perinatal':       dict(),
        'neonatal':        dict(),
        'early_neonatal':  dict(),
        'late_neonatal':   dict(),
    }
    # мертворожденные дети
    df_stillbirth = df[df['lifetime'] == 0]
    
    # дети умершие в перинатальном возрасте
    df_perinatal = df[df['lifetime'] <= 7]
    
    # дети умершие в неонатальном возрасте
    df_neonatal = df[(df['lifetime'] <= 28) &
                     (df['lifetime'] > 0)]
    
    # дети умершие в раннем неонатальном возрасте
    df_early_neonatal = df[(df['lifetime'] <= 7) &
                           (df['lifetime'] > 0)]
    
    # дети умершие в позднем неонатальном возрасте
    df_late_neonatal = df[(df['lifetime'] <= 28) &
                          (df['lifetime'] > 7)]

    # дети родившиеся живыми
    df_born_alive = df[~df.index.isin(df_stillbirth.index)]
    
    for year in tqdm(years):
        
        # количество рожденных живыми и мертвыми в год year
        total_born = df[
            df.birth_date.dt.year == year
        ].shape[0]
        
        # количество рожденных живыми в год year
        total_born_alive = df_born_alive[
            df_born_alive.birth_date.dt.year == year
        ].shape[0]
        
        rates['stillbirth'][year] = df_stillbirth[
                df_stillbirth.birth_date.dt.year == year
            ].shape[0] / total_born
        
        rates['perinatal'][year] = df_perinatal[
                df_perinatal.birth_date.dt.year == year
            ].shape[0] / total_born
        
        rates['neonatal'][year] = df_neonatal[
                df_neonatal.birth_date.dt.year == year
            ].shape[0] / total_born_alive
        
        rates['early_neonatal'][year] = df_early_neonatal[
                df_early_neonatal.birth_date.dt.year == year
            ].shape[0] / total_born_alive

        rates['late_neonatal'][year] = df_late_neonatal[
                df_late_neonatal.birth_date.dt.year == year
            ].shape[0] / total_born_alive
    
    df_infant_mortality = pd.DataFrame.from_dict(rates)
    
    return df_infant_mortality

In [30]:
# сохраняем в таблицу
df_infant_mortality = infant_mortality(df, 1950, 2021)
df_infant_mortality.to_csv('results/infant_mortality.csv')
df_infant_mortality

100%|██████████| 71/71 [00:06<00:00, 11.63it/s]


,stillbirth,perinatal,neonatal,early_neonatal,late_neonatal
1950,0.006397,0.008048,0.004569,0.001661,0.002908
1951,0.004425,0.006235,0.003838,0.001818,0.002020
1952,0.005859,0.008788,0.005893,0.002947,0.002947
1953,0.004706,0.007571,0.007607,0.002878,0.004729
1954,0.004719,0.006771,0.005978,0.002061,0.003917
...,...,...,...,...,...
2016,0.005943,0.007582,0.006597,0.001649,0.004947
2017,0.005492,0.007120,0.004909,0.001636,0.003273
2018,0.005699,0.007735,0.005322,0.002047,0.003275
2019,0.005490,0.009412,0.006703,0.003943,0.002760
